In [0]:
import random
import numpy as np
import pandas as pd

import torch
import torchvision
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
!wget 'https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_features.csv'
!wget 'https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_labels.csv'
!wget 'https://s3.amazonaws.com/drivendata-prod/data/66/public/test_set_features.csv'

--2020-06-13 13:28:15--  https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_features.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.177.205
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.177.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4619138 (4.4M) [text/csv]
Saving to: ‘training_set_features.csv’

training_set_featur 100%[===================>]   4.40M  5.90MB/s    in 0.7s    

2020-06-13 13:28:16 (5.90 MB/s) - ‘training_set_features.csv’ saved [4619138/4619138]

--2020-06-13 13:28:17--  https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_labels.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.9.214
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.9.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256004 (250K) [text/csv]
Saving to: ‘training_set_labels.csv’

training_set_labels 100%[===================>] 250.00K   673KB/s    in 0.4s    

2020-06-

In [0]:
training_data = pd.read_csv("training_set_features.csv")

In [4]:
training_data

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,1.0,5.0,2.0,2.0,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,4.0,2.0,5.0,4.0,2.0,55 - 64 Years,Some College,White,Female,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,18 - 34 Years,Some College,Hispanic,Female,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg


In [0]:
training_data = training_data.drop(["health_insurance", "employment_industry", "employment_occupation"], axis=1)

In [0]:
def convert2numb(data):
    data["marital_status"] = data["marital_status"].map({"Married": 1, "Not Married": 2})
    data["race"] = data["race"].map({"White": 1, "Black": 2, "Hispanic": 3, 
                                                       "Other or Multiple": 4})
    data["sex"] = data["sex"].map({"Male": 1, "Female": 2})
    data["rent_or_own"] = data["rent_or_own"].map({"Own": 1, "Rent": 2})
    data["employment_status"] = data["employment_status"].map({"Employed": 1,
                                                                            "Not in Labor Force": 2, 
                                                                            "Unemployed": 3})
    data["income_poverty"] = data["income_poverty"].map({"<= $75,000, Above Poverty": 1, 
                                                                     "> $75,000": 2,
                                                                     "Below Poverty": 3})
    data["hhs_geo_region"] = data["hhs_geo_region"].map({"lzgpxyit": 1, "fpwskwrf": 2, 
                                                                       "qufhixun": 3, "oxchjgsf": 4,
                                                                       "kbazzjca": 5, "bhuqouqj": 6,
                                                                       "mlyzmhmf": 7, "lrircsnp": 8,
                                                                       "atmpeygn": 9, "dqpwygqj": 10}) 
    data["census_msa"] = data["census_msa"].map({"MSA, Not Principle  City": 1, 
                                                               "MSA, Principle City": 2, "Non-MSA": 3})
    data["age_group"] = data["age_group"].map({"65+ Years": 1, "55 - 64 Years": 2, 
                                                             "45 - 54 Years": 3, "18 - 34 Years": 4, 
                                                             "35 - 44 Years": 5})
    data["education"] = data["education"].map({"College Graduate": 1, "Some College": 2, 
                                                             "12 Years": 3, "< 12 Years": 4})

In [0]:
convert2numb(training_data)

In [8]:
training_data

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,2,4.0,1,2,3.0,2.0,1.0,2.0,4,3,0.0,0.0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,4.0,4.0,2.0,4.0,5,3.0,1,1,3.0,2.0,2.0,1.0,6,1,0.0,0.0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,3.0,1.0,1.0,4.0,1.0,2.0,4,1.0,1,1,1.0,2.0,1.0,1.0,3,1,2.0,0.0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,3.0,5.0,5.0,4.0,1.0,1,3.0,1,2,3.0,2.0,2.0,2.0,8,2,0.0,0.0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,3.0,1.0,4.0,3,2.0,1,2,1.0,1.0,1.0,1.0,3,1,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,5.0,2.0,2.0,1,2.0,1,2,1.0,2.0,1.0,2.0,3,3,0.0,0.0
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,4,1.0,1,1,1.0,2.0,2.0,1.0,1,2,1.0,0.0
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,2.0,5.0,4.0,2.0,2,2.0,1,2,NaN,2.0,1.0,NaN,1,1,0.0,0.0
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,4,2.0,3,2,1.0,1.0,2.0,1.0,8,3,1.0,0.0


In [9]:
training_data.isnull().sum()

respondent_id                     0
h1n1_concern                     92
h1n1_knowledge                  116
behavioral_antiviral_meds        71
behavioral_avoidance            208
behavioral_face_mask             19
behavioral_wash_hands            42
behavioral_large_gatherings      87
behavioral_outside_home          82
behavioral_touch_face           128
doctor_recc_h1n1               2160
doctor_recc_seasonal           2160
chronic_med_condition           971
child_under_6_months            820
health_worker                   804
opinion_h1n1_vacc_effective     391
opinion_h1n1_risk               388
opinion_h1n1_sick_from_vacc     395
opinion_seas_vacc_effective     462
opinion_seas_risk               514
opinion_seas_sick_from_vacc     537
age_group                         0
education                      1407
race                              0
sex                               0
income_poverty                 4423
marital_status                 1408
rent_or_own                 

In [0]:
def give_number(data, col):
    numbs = data[col].value_counts().index.tolist()
    weighted_random = []
    for item in numbs:
        weighted_random += [item]*(data[col].values == item).sum()
    return weighted_random

In [0]:
cols_with_na = []
for col in training_data.columns:
    if training_data[col].isna().any():
        cols_with_na.append(col)

In [12]:
for col in cols_with_na:
    weighted_random = give_number(training_data, col)
    for i in range(len(training_data[col])):
        if pd.isna(training_data[col][i]):
            training_data[col][i] = random.choice(weighted_random)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
training_data.isna().sum()

respondent_id                  0
h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [33]:
labels = pd.read_csv("training_set_labels.csv")
labels

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0


In [0]:
train_df = pd.merge(training_data, labels, left_on="respondent_id", right_on="respondent_id")

In [16]:
train_df

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,2,4.0,1,2,3.0,2.0,1.0,2.0,4,3,0.0,0.0,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,4.0,4.0,2.0,4.0,5,3.0,1,1,3.0,2.0,2.0,1.0,6,1,0.0,0.0,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,1.0,1.0,4.0,1.0,2.0,4,1.0,1,1,1.0,2.0,1.0,1.0,3,1,2.0,0.0,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,3.0,5.0,5.0,4.0,1.0,1,3.0,1,2,3.0,2.0,2.0,2.0,8,2,0.0,0.0,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,3.0,1.0,4.0,3,2.0,1,2,1.0,1.0,1.0,1.0,3,1,1.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,5.0,2.0,2.0,1,2.0,1,2,1.0,2.0,1.0,2.0,3,3,0.0,0.0,0,0
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,4,1.0,1,1,1.0,2.0,2.0,1.0,1,2,1.0,0.0,0,0
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,2.0,5.0,4.0,2.0,2,2.0,1,2,2.0,2.0,1.0,2.0,1,1,0.0,0.0,0,1
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,4,2.0,3,2,1.0,1.0,2.0,1.0,8,3,1.0,0.0,0,0


In [0]:
features = train_df.columns.difference(["respondent_id","h1n1_vaccine", "seasonal_vaccine"])
targets = ["h1n1_vaccine", "seasonal_vaccine"]

In [0]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df[features], 
    train_df[targets],
    test_size = 0.1, 
    random_state=42)

In [39]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(24036, 32)
(2671, 32)
(24036, 2)
(2671, 2)


In [40]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

# Training

In [0]:
h1_n1_train_ds = TensorDataset(torch.from_numpy(x_train.values).float(), 
                               torch.from_numpy(y_train["h1n1_vaccine"].values).long())
h1_n1_train_dl = DataLoader(h1_n1_train_ds, batch_size=1024, shuffle=True)

h1_n1_valid_ds = TensorDataset(torch.from_numpy(x_valid.values).float(), 
                               torch.from_numpy(y_valid["h1n1_vaccine"].values).long())
h1_n1_valid_dl = DataLoader(h1_n1_valid_ds, batch_size=1024, shuffle=True)

In [0]:
seasonal_vaccine_train_ds = TensorDataset(torch.from_numpy(x_train.values).float(), 
                                          torch.from_numpy(y_train["seasonal_vaccine"].values).long())
seasonal_vaccine_train_dl = DataLoader(seasonal_vaccine_train_ds, batch_size=1024, shuffle=True)

seasonal_vaccine_valid_ds = TensorDataset(torch.from_numpy(x_valid.values).float(), 
                                          torch.from_numpy(y_valid["seasonal_vaccine"].values).long())
seasonal_vaccine_valid_dl = DataLoader(seasonal_vaccine_valid_ds, batch_size=1024, shuffle=True)

In [0]:
def h1_n1_create_model():
    model = nn.Sequential(
        nn.Linear(32, 20),
        nn.ReLU(),
        # nn.Dropout(0.5),
        nn.Linear(20, 16),
        nn.ReLU(),
        # nn.Dropout(0.5),
        nn.Linear(16, 8),
        nn.ReLU(),
        # nn.Dropout(0.5),
        nn.Linear(8, 4),
        nn.ReLU(),
        nn.Linear(4, 2),
        nn.ReLU()
#         nn.Sigmoid()
    )
    return model

In [0]:
def seasonal_vaccine_create_model():
    model = nn.Sequential(
        # nn.Linear(60, 32),
        # nn.ReLU(),
        # nn.Dropout(0.5),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(16, 2),
#         nn.Sigmoid()
    )
    return model

In [0]:
def validate(valid_dl, model, device=device):
    losses = []
    loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.25, 1]).to(device))
    total = 0
    correct = 0
    for i, (inputs, labels) in enumerate(valid_dl):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            pred = model(inputs)
            loss = loss_fn(pred, labels)
            losses.append(loss.item())
            

    losses = np.array(losses)
    return np.mean(losses)

In [0]:
def train(string, train_dl, valid_dl, epochs=10, learning_rate=1e-3, device=device):
    if string == "h1_n1":
        model = h1_n1_create_model().to(device)
    else:
        model = seasonal_vaccine_create_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.25, 1]).to(device))
    for epoch in range(epochs):
        losses = []
        count = 0
        for i, (inputs, labels) in enumerate(train_dl, 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            pred = model(inputs)
            loss = loss_fn(pred, labels)
            with torch.no_grad():
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            losses.append(loss.item() * len(labels))
            count += len(labels)
        val_loss = validate(valid_dl, model, device=device)
        if epoch % 10 == 0:
            print(f"Epoch: {epoch} Training Loss: {sum(losses)/count:.3f} Val Loss: {val_loss}")
    return model

In [48]:
h1_n1_model = train("h1_n1", h1_n1_train_dl, h1_n1_valid_dl, epochs=500, learning_rate=1e-3, device=device)

Epoch: 0 Training Loss: 0.695 Val Loss: 0.6914425094922384
Epoch: 10 Training Loss: 0.530 Val Loss: 0.5360611875851949
Epoch: 20 Training Loss: 0.517 Val Loss: 0.5172732273737589
Epoch: 30 Training Loss: 0.514 Val Loss: 0.509689062833786
Epoch: 40 Training Loss: 0.511 Val Loss: 0.5121419628461202
Epoch: 50 Training Loss: 0.510 Val Loss: 0.527273694674174
Epoch: 60 Training Loss: 0.509 Val Loss: 0.5102712512016296
Epoch: 70 Training Loss: 0.505 Val Loss: 0.5098769863446554
Epoch: 80 Training Loss: 0.503 Val Loss: 0.5061131616433462
Epoch: 90 Training Loss: 0.502 Val Loss: 0.5123865306377411
Epoch: 100 Training Loss: 0.500 Val Loss: 0.51052192846934
Epoch: 110 Training Loss: 0.500 Val Loss: 0.5086689094702402
Epoch: 120 Training Loss: 0.499 Val Loss: 0.5084884762763977
Epoch: 130 Training Loss: 0.498 Val Loss: 0.5099023381868998
Epoch: 140 Training Loss: 0.497 Val Loss: 0.509671817223231
Epoch: 150 Training Loss: 0.497 Val Loss: 0.5044883489608765
Epoch: 160 Training Loss: 0.497 Val Loss

In [0]:
seasonal_vaccine_model = train("seasonal_vaccine", seasonal_vaccine_train_dl, seasonal_vaccine_valid_dl, epochs=500, learning_rate=1e-2, device=device)

# Save Model

In [0]:
torch.save(h1_n1_model.state_dict(), "h1_n1.pth")

In [0]:
torch.save(seasonal_vaccine_model.state_dict(), "seasonal_vaccine.pth")

# Loading Model

In [0]:
h1_n1_model = h1_n1_create_model()
h1_n1_model.load_state_dict(torch.load("h1_n1.pth"))
h1_n1_model.eval()

Sequential(
  (0): Linear(in_features=60, out_features=32, bias=True)
  (1): Tanh()
  (2): Dropout(p=0.25, inplace=False)
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): Tanh()
  (5): Dropout(p=0.25, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
  (7): Sigmoid()
)

In [0]:
seasonal_vaccine_model = seasonal_vaccine_create_model()
seasonal_vaccine_model.load_state_dict(torch.load("seasonal_vaccine.pth"))
seasonal_vaccine_model.eval()

Sequential(
  (0): Linear(in_features=60, out_features=32, bias=True)
  (1): Tanh()
  (2): Dropout(p=0.25, inplace=False)
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): Tanh()
  (5): Dropout(p=0.25, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
  (7): Sigmoid()
)

In [0]:
validate(h1_n1_valid_dl, h1_n1_model.to(device), device=device)

0.5916744470596313

In [0]:
validate(seasonal_vaccine_valid_dl, seasonal_vaccine_model.to(device), device=device)

0.41251445810000104

# Testing Data

In [0]:
test_data = pd.read_csv("test_set_features_clean.csv")
ids = test_data["respondent_id"]
# test_data = test_data.drop(["health_insurance", "employment_industry", "employment_occupation"], axis=1)

In [0]:
ids

0        26707
1        26708
2        26709
3        26710
4        26711
         ...  
26703    53410
26704    53411
26705    53412
26706    53413
26707    53414
Name: respondent_id, Length: 26708, dtype: int64

In [0]:
test_data

,respondent_id,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,...,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
0,26707,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0,0,0,0,1,0,0,1,0,0
1,26708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,26709,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1
3,26710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,0,1,0,0
4,26711,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,1,0
26704,53411,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
26705,53412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,1,0,0
26706,53413,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [0]:
test_data.isnull().sum()

respondent_id                          0
behavioral_antiviral_meds              0
behavioral_avoidance                   0
behavioral_face_mask                   0
behavioral_wash_hands                  0
                                      ..
hhs_geo_region_oxchjgsf                0
hhs_geo_region_qufhixun                0
census_msa_MSA, Not Principle  City    0
census_msa_MSA, Principle City         0
census_msa_Non-MSA                     0
Length: 61, dtype: int64

In [0]:
# cols_with_na = []
# for col in test_data.columns:
#     if test_data[col].isna().any():
#         cols_with_na.append(col)

In [0]:
# for col in cols_with_na:
#     weighted_random = give_number(test_data, col)
#     for i in range(len(test_data[col])):
#         if pd.isna(test_data[col][i]):
#             test_data[col][i] = random.choice(weighted_random)

In [0]:
test_data.isnull().sum()

respondent_id                          0
behavioral_antiviral_meds              0
behavioral_avoidance                   0
behavioral_face_mask                   0
behavioral_wash_hands                  0
                                      ..
hhs_geo_region_oxchjgsf                0
hhs_geo_region_qufhixun                0
census_msa_MSA, Not Principle  City    0
census_msa_MSA, Principle City         0
census_msa_Non-MSA                     0
Length: 61, dtype: int64

In [0]:
test_data = test_data.drop("respondent_id", axis=1).values
test_data

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [0]:
test_data.shape

(26708, 60)

In [0]:
test_ds = TensorDataset(torch.from_numpy(test_data).float(), torch.ones(test_data.shape[0]))
test_dl = DataLoader(test_ds, batch_size=26708, shuffle=False)

In [0]:
test_dl

In [0]:
for i, (inputs, _) in enumerate(test_dl):
    print(i)
    h1_n1_model = h1_n1_model.to(device)
    inputs = inputs.to(device)
    with torch.no_grad():
        h1_n1_pred = h1_n1_model(inputs)
        seasonal_vaccine_model = seasonal_vaccine_model.to(device)
        seasonal_vaccine_pred = seasonal_vaccine_model(inputs)

0


In [0]:
(h1_n1_pred < 0.1).sum()

tensor(28059, device='cuda:0')

In [0]:
h1_n1_pred.cpu().numpy().flatten()

array([ 35.62537   , -40.302902  ,   5.1693225 , ...,  -0.16711389,
         4.5046706 ,  -6.346066  ], dtype=float32)

In [0]:
print(seasonal_vaccine_pred.shape)
print(h1_n1_pred.shape)

torch.Size([26708, 2])
torch.Size([26708, 2])


In [0]:
h1_n1_pred[0]

tensor([ 35.6254, -40.3029])

In [0]:
seasonal_vaccine_pred = seasonal_vaccine_pred.cpu()

In [0]:
submission = pd.DataFrame({'respondent_id':ids, 
                           "h1n1_vaccine": h1_n1_pred.cpu().numpy().flatten(),
                           "seasonal_vaccine": seasonal_vaccine_pred.cpu().numpy().flatten()
                          })
submission = submission[['respondent_id', 'h1n1_vaccine', 'seasonal_vaccine']]
submission

ValueError: array length 53416 does not match index length 26708

In [0]:
submission.to_csv("submission.csv", index=False)